![](https://i.imgur.com/JsMvGnP.png)

# Projeto - Classificação

## 1. Considerações iniciais

Neste notebook iremos praticar a construção de um modelo de Machine Learning. Na atividade, iremos utilizar um algoritmo para realizar um processo de classificação. 

Como o foco da atividade será percorrer o fluxo do Aprendizado de Máquina por inteiro (ou o máximo dele), evitaremos, propositalmente, adentrar em algoritmos e explicações mais complexas.


### 1.1. O Problema de Negócio

A _churn rate_ (taxa de rotatividade ou cancelamento) é a porcentagem de assinantes que interrompem suas assinaturas de um serviço dentro de um determinado período de tempo. Para uma empresa expandir sua clientela, sua taxa de crescimento, medida pelo número de novos clientes, deve exceder sua taxa de cancelamento (_churn rate_).

Portanto, neste projeto, um banco deseja que você construa um modelo de Machine Learning que possa prever a probabilidade de seus clientes atuais fecharem suas contas em um futuro próximo e, portanto, calcular sua _churn rate_.

Esse é um problema de classificação no qual você classifica se um cliente irá sair (y=1) ou permanecer (y=0) com base diversos atributos.

### 1.2. Descrição do dados

Os dados necessários podem ser encontrados no [repositório da atividade](https://github.com/awarischool/br-classification-project). No arquivo *Churn_Modelling.csv* você encontrará:
- **RowNumber**: id do registro/linha.
- **CustomerId**: identificador único do cliente.
- **Surname**: sobrenome do cliente.
- **CreditScore**: pontuação de crédito do cliente.
- **Geography**: país de origem do cliente.
- **Gender**: sexo do cliente.
- **Age**: idade do cliente.
- **Tenure**: número de anos em que o cliente está com o banco.
- **Balance**: saldo bancário.
- **NumOfProducts**: número de produtos que o cliente utiliza.
- **HasCrCard**: indica se o cliente tem (1) ou não tem (0) cartão de crédito.
- **IsActiveMember**: indica se o cliente é (1) ou não é (0) ativo no banco.
- **EstimatedSalary**: salário estimado em dólares.
- **Exited**: indica se o cliente encerrou (1) a conta no banco ou se o cliente foi retido (0).


### 1.3 Métricas de Avaliação

Avalie seu modelo utilizando as métricas Acurácia e AUC (area under the ROC) estudadas nos módulos anteriores.

Trabalhe até conseguir uma acurácia acima e 85%.

## 2. Etapas do Projeto

Neste projeto você poderá seguir o fluxo que preferir, mas como sugestão pode seguir as seguintes etapas:
1. Análise exploratória 
2. Preparação e limpeza dos dados
3. Seleção de atributos
4. Construção do modelo
5. Avaliação do modelo
6. Otimização do modelo

Lembrando que esses passos, provavelmente, serão executados várias vezes até se obter um resultado satisfatório.

Sugestões de modelos:
* Logistic Regression Classification
* K-Nearest Neighbour (KNN) Classification
* Support Vector Machine (SVM) Classification
* Naive Bayes Classification
* Decision Tree Classification
* Random Forest Classification

Em caso de dúvidas, não esqueça de conversar com seu mentor.

## 3. Desenvolvimento do Projeto

In [52]:
# Importando bibliotecas necessárias para o projeto

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 3.1 Análise exploratória

In [53]:
# Carregando conjunto de dados.

url = 'https://raw.githubusercontent.com/viniciusacosta/br-classification-project/master/Churn_Modelling.csv'
churn = pd.read_csv(url)

In [54]:
# Exibindo as primeiras linhas do DataFrame

churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [55]:
# Transformando todas as letras em minúsculas.
churn.columns = churn.columns.str.lower()

# Resultado da transformação
churn.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [56]:
# Verificando as dimensões do dataset.

churn.shape

(10000, 14)

Verificamos a existência de **14 variáveis**, e **10000 observações** dentro do dataset.

In [57]:
# Verificando o número de registros duplicados.

churn.duplicated().sum()

0

**Não há registros duplicados** dentro do conjunto de dados.

In [58]:
# Verificando o número de NAs existentes dentro do dataset.

churn.isna().sum()

rownumber          0
customerid         0
surname            0
creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64

**Não há valores nulos** dentro do conjunto de dados.

In [59]:
# Contabilizando o número de valores únicos em cada variável do dataset.
info = churn.nunique().sort_values()

# Determinando o tipo de dado de cada uma das variáveis do dataset.
info = pd.DataFrame(info.values, index = info.index, columns = ['NUniques'])

# Atribuindo informações sobre o tipo de dado das variáveis ao DataFrame.
info['dtypes'] = churn.dtypes

# Exibindo Dataframe.
info

,NUniques,dtypes
gender,2,object
hascrcard,2,int64
isactivemember,2,int64
exited,2,int64
geography,3,object
numofproducts,4,int64
tenure,11,int64
age,70,int64
creditscore,460,int64
surname,2932,object


Todas as variáveis preditoras apresentam um grande número de valores únicos.

In [60]:
# Eliminando a variáveis desnecessárias do conjunto de dados.

churn = churn.drop(columns = ['rownumber', 'customerid', 'surname'])

In [62]:
# OneHotEncoding do atributo geography
churn = pd.get_dummies(churn, columns=['geography'])
churn.columns = churn.columns.str.lower()

# Transformando coluna gender para valores númericos
churn['gender'] = churn['gender'].map({'Female':1, 'Male':0})

In [63]:
# Verificando dados estatísticos descritivos

churn.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
creditscore,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
gender,10000.0,0.45,0.50,0.00,0.00,0.00,1.00,1.00
age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
tenure,10000.0,5.01,2.89,0.00,3.00,5.00,7.00,10.00
balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
numofproducts,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
hascrcard,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
isactivemember,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
estimatedsalary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48
exited,10000.0,0.20,0.40,0.00,0.00,0.00,0.00,1.00


De acordo com a tabela, podemos perceber uma possivel presença de outliers na variável **balance**.

In [64]:
# Criar boxplot da variável saldo_bancario
sns.displot(df['saldo_bancario']);

NameError: ignored

In [ ]:
# Separando as features categóricas das númericas
continuous_var = ['creditscore', 'age', 'tenure', 'balance', 'estimatedsalary']
categorical_var = ['geography_france','geography_germany', 'geography_spain',  'gender', 'hascrcard', 'numofproducts', 'isactivemember']

### 3.2 Preparação e limpeza dos dados

In [ ]:
# separate categorical and numerical churn
categorical_features = ['Geography', 'Gender', 'NumOfProducts', 'HasCrCard',  'IsActiveMember']
features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

### 3.3 Seleção de atributos

In [ ]:
# seu código

In [ ]:
# seu código

### 3.4 Construção do modelo

In [ ]:
# seu código

In [ ]:
# seu código

### 3.5 Avaliação do modelo

In [ ]:
# seu código

In [ ]:
# seu código

### 3.4 Otimização do modelo

In [ ]:
# seu código

In [ ]:
# seu código

## 4. Conclusão
O fluxo de trabalho de Aprendizado de Máquina não termina após a avaliação de performance do modelo escolhido. Na verdade, esta seria somente a primeira iteração de um processo que pode ter dezenas de tentativas.

Encorajamos você a repetir novamente o processo descrito neste notebook. Contudo, explorando novas possibilidades para verificar a melhora/piora da performance do seu modelo. 

Você pode, por exemplo, remover variáveis/atributos menos importantes, normalizar variáveis importantes e escolher outros algoritmos de aprendizagem supervisionada. O Aprendizado de Máquina é uma área interessante justamente por causa disto. Você testa, você erra, você recomeça.